# Projeto: Segmentação de Vasos Sanguíneos da Retina com **U‑Net++**

> Guia completo para implementar e treinar um modelo **U‑Net++** no dataset **DRIVE**, capaz de detectar com precisão vasos sanguíneos em imagens de fundo de olho ─ tarefa fundamental no diagnóstico precoce de doenças como a retinopatia diabética.

---

## 1  ·  Visão Geral e Objetivos
| Item          | Escolha |
|---------------|---------|
| **Dataset**   | DRIVE (Digital Retinal Images for Vessel Extraction) |
| **Modelo**    | U‑Net++ |
| **Framework** | TensorFlow / Keras |
| **Ambiente**  | Kaggle Notebook com GPU (T4 × 2) |

*Desafio:* Criar um pipeline de **deep learning** que segmente vasos sanguíneos com alta sensibilidade e especificidade.

---

## 2. Compreendendo a U-Net++

Antes de codificar, é crucial entender por que a U-Net++ é uma melhoria em relação à U-Net original.

A U-Net introduziu as skip connections (conexões de atalho), que combinam características de mapas de alta resolução (do codificador) com mapas de características reamostrados (do decodificador). Isso ajuda a recuperar detalhes espaciais perdidos durante o downsampling.

A U-Net++ aprimora isso com duas inovações principais:

- **Conexões de Atalho Aninhadas e Densas (Nested and Dense Skip Connections):** Em vez de uma única conexão longa, a U-Net++ insere nós de convolução ao longo das skip connections. Isso cria um caminho mais gradual para o fluxo de gradientes e preenche a "lacuna semântica" entre os mapas de características do codificador e do decodificador. Os mapas de características que chegam ao decodificador são semanticamente mais ricos.

- **Supervisão Profunda (Deep Supervision):** O modelo pode ser treinado para gerar saídas em múltiplas resoluções. As saídas dos nós X_0,1, X_0,2, X_0,3 e X_0,4 (na notação do paper) podem ser usadas para calcular uma perda combinada. Isso força os nós intermediários a aprenderem características úteis, resultando em uma otimização mais robusta e a capacidade de "podar" o modelo em tempo de inferência para um equilíbrio entre velocidade e precisão.


## 3. Configuração do Ambiente Kaggle


Na primeira célula, vamos instalar e importar todas as bibliotecas necessárias e carregar o dataset usando kagglehub.

In [ ]:
# Instalação do Kaggle Hub se não estiver presente
!pip install -q kagglehub

# Importações essenciais
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import matplotlib.pyplot as plt
import cv2 # OpenCV para algumas operações de pré-processamento
from sklearn.model_selection import train_test_split

# Import para carregar o dataset
import kagglehub

# Silenciar logs desnecessários
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
# Download e descompactação do dataset
# O Kaggle gerencia o caminho automaticamente
dataset_path = kagglehub.dataset_download("andrewmvd/drive-digital-retinal-images-for-vessel-extraction")

print(f"Dataset baixado em: {dataset_path}")

# Definir os caminhos principais baseados na estrutura do dataset
DRIVE_DIR = os.path.join(dataset_path, "DRIVE")
TRAIN_IMG_DIR = os.path.join(DRIVE_DIR, "training/images")
TRAIN_MASK_DIR = os.path.join(DRIVE_DIR, "training/1st_manual") # Usaremos a anotação manual como ground truth
TRAIN_FOV_DIR = os.path.join(DRIVE_DIR, "training/mask") # Máscaras do campo de visão (FOV)

TEST_IMG_DIR = os.path.join(DRIVE_DIR, "test/images")
TEST_MASK_DIR = os.path.join(DRIVE_DIR, "test/1st_manual") # Disponível no Kaggle, mas vamos simular um teste real
TEST_FOV_DIR = os.path.join(DRIVE_DIR, "test/mask")

### Pastas do dataset
```
DRIVE/
├── training/
│   ├── mask/                       # Contém máscaras de treinamento em formato GIF
│   │   ├── 21_training_mask.gif
│   │   ├── 22_training_mask.gif
│   │   └── ... (até 40_training_mask.gif)
│   ├── images/                     # Contém imagens de treinamento em formato TIF
│   │   ├── 21_training.tif
│   │   ├── 22_training.tif
│   │   └── ... (até 40_training.tif)
│   └── 1st_manual/                 # Contém anotações manuais em formato GIF
│       ├── 21_manual1.gif
│       ├── 22_manual1.gif
│       └── ... (até 40_manual1.gif)
└── test/
    ├── mask/                       # Contém máscaras de teste em formato GIF
    │   ├── 01_test_mask.gif
    │   ├── 02_test_mask.gif
    │   └── ... (até 20_test_mask.gif)
    └── images/                     # Contém imagens de teste em formato TIF
        ├── 01_test.tif
        ├── 02_test.tif
        └── ... (até 20_test.tif)
```


## 4  ·  Carregamento e Pré-processamento dos Dados

Imagens médicas requerem um pré-processamento criterioso. Isso inclui normalização, redimensionamento e, opcionalmente, divisão em **patches** (recortes menores), o que ajuda a aumentar a eficiência e a variabilidade do treinamento.

---

### 4.1  ·  Funções de Carregamento

A seguir, criaremos uma função que:

- Lê imagens `.tif` e máscaras `.gif`
- Converte para arrays NumPy
- Normaliza os valores para o intervalo **[0, 1]**

In [ ]:
def load_data(img_dir, mask_dir, fov_dir):
    """Carrega imagens, máscaras e máscaras de FOV."""
    img_files = sorted(glob.glob(os.path.join(img_dir, "*.tif")))
    mask_files = sorted(glob.glob(os.path.join(mask_dir, "*.gif")))
    fov_files = sorted(glob.glob(os.path.join(fov_dir, "*.gif")))

    images = []
    masks = []
    fov_masks = []

    for img_path, mask_path, fov_path in zip(img_files, mask_files, fov_files):
        # Carregar imagem e normalizar
        img = Image.open(img_path).convert('RGB')
        img = np.array(img, dtype=np.float32) / 255.0
        images.append(img)

        # Carregar máscara e binarizar
        mask = Image.open(mask_path)
        mask = np.array(mask, dtype=np.float32)
        mask[mask > 0] = 1.0 # Garantir que seja 0 ou 1
        mask = np.expand_dims(mask, axis=-1) # Adicionar canal
        masks.append(mask)

        # Carregar máscara de FOV e binarizar
        fov = Image.open(fov_path)
        fov = np.array(fov, dtype=np.float32)
        fov[fov > 0] = 1.0
        fov = np.expand_dims(fov, axis=-1)
        fov_masks.append(fov)

    return np.array(images), np.array(masks), np.array(fov_masks)

# Carregar os dados de treinamento
train_images, train_masks, train_fov_masks = load_data(TRAIN_IMG_DIR, TRAIN_MASK_DIR, TRAIN_FOV_DIR)

print(f"Imagens de treinamento carregadas: {train_images.shape}")
print(f"Máscaras de treinamento carregadas: {train_masks.shape}")

---

### 4.2  ·  Criação de Patches (Recortes)

As imagens originais (584×565) são grandes demais para caber na memória da GPU de uma só vez.  
A técnica padrão é extrair **patches** menores e treinar o modelo com eles.


In [ ]:
def create_patches(images, masks, patch_size=128, stride=64):
    """Extrai patches sobrepostos das imagens e máscaras."""
    img_patches = []
    mask_patches = []

    img_h, img_w = images.shape[1], images.shape[2]

    for i in range(images.shape[0]): # Para cada imagem
        for y in range(0, img_h - patch_size + 1, stride):
            for x in range(0, img_w - patch_size + 1, stride):
                img_patch = images[i, y:y+patch_size, x:x+patch_size]
                mask_patch = masks[i, y:y+patch_size, x:x+patch_size]
                
                # Adicionar apenas patches que contenham alguma informação de vaso
                if np.sum(mask_patch) > 0:
                    img_patches.append(img_patch)
                    mask_patches.append(mask_patch)

    return np.array(img_patches), np.array(mask_patches)

# Criar patches para treinamento e validação
# Usaremos um split simples para criar um conjunto de validação
X_train, X_val, y_train, y_val = train_test_split(train_images, train_masks, test_size=0.1, random_state=42)

train_img_patches, train_mask_patches = create_patches(X_train, y_train)
val_img_patches, val_mask_patches = create_patches(X_val, y_val)

print(f"Patches de imagem de treinamento: {train_img_patches.shape}")
print(f"Patches de máscara de treinamento: {train_mask_patches.shape}")
print(f"Patches de imagem de validação: {val_img_patches.shape}")
print(f"Patches de máscara de validação: {val_mask_patches.shape}")

In [ ]:
import matplotlib.pyplot as plt

# Definir o número de exemplos que queremos visualizar
num_samples_to_show = 3

print("Exibindo exemplos de imagens de treino e suas máscaras correspondentes...")

# Criar uma figura para cada par de imagem/máscara
for i in range(num_samples_to_show):
    
    # Criar uma nova figura com tamanho adequado
    plt.figure(figsize=(12, 5))
    
    # Plotar a Imagem Original
    plt.subplot(1, 2, 1)
    plt.title(f"Imagem de Treino #{i+1}")
    plt.imshow(X_train[i]) # X_train contém as imagens originais
    plt.axis('off') # Remover os eixos para uma visualização mais limpa
    
    # Plotar a Máscara (Gabarito)
    plt.subplot(1, 2, 2)
    plt.title(f"Máscara (Gabarito) #{i+1}")
    plt.imshow(y_train[i], cmap='gray') # y_train contém as máscaras. 'cmap=gray' garante a exibição em preto e branco.
    plt.axis('off')
    
    # Mostrar o par de plots
    plt.show()

---

## 5  ·  Implementação da U‑Net++

Agora, a parte central: **construir o modelo**.  
Vamos criar um **bloco de convolução padrão** e depois montar a arquitetura **U‑Net++** completa.


In [ ]:
# Bloco de convolução padrão
def conv_block(inputs, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    return x

# Função para construir o modelo U-Net++
def build_unet_plus_plus(input_shape, deep_supervision=False):
    """
    Constrói a arquitetura U-Net++.
    Notação: X_i_j onde i é o nível de downsampling e j é a camada de convolução no skip path.
    """
    inputs = layers.Input(shape=input_shape)
    
    # --- Codificador ---
    X_0_0 = conv_block(inputs, 32)
    p0 = layers.MaxPooling2D(2)(X_0_0)
    
    X_1_0 = conv_block(p0, 64)
    p1 = layers.MaxPooling2D(2)(X_1_0)

    X_2_0 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D(2)(X_2_0)
    
    X_3_0 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D(2)(X_3_0)
    
    X_4_0 = conv_block(p3, 512) # Camada bottleneck

    # --- Skip Pathways Aninhados e Densos ---
    
    # Nível 1
    u1_0 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(X_1_0)
    X_0_1 = conv_block(layers.concatenate([X_0_0, u1_0]), 32)
    
    # Nível 2
    u2_0 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(X_2_0)
    X_1_1 = conv_block(layers.concatenate([X_1_0, u2_0]), 64)

    u1_1 = layers.Conv2DTranspose(32, 2, strides=2, padding="same")(X_1_1)
    X_0_2 = conv_block(layers.concatenate([X_0_0, X_0_1, u1_1]), 32)
    
    # Nível 3
    u3_0 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(X_3_0)
    X_2_1 = conv_block(layers.concatenate([X_2_0, u3_0]), 128)
    
    u2_1 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(X_2_1)
    X_1_2 = conv_block(layers.concatenate([X_1_0, X_1_1, u2_1]), 64)
    
    u1_2 = layers.Conv2DTranspose(32, 2, strides=2, padding="same")(X_1_2)
    X_0_3 = conv_block(layers.concatenate([X_0_0, X_0_1, X_0_2, u1_2]), 32)
    
    # Nível 4 (Decoder)
    u4_0 = layers.Conv2DTranspose(512, 2, strides=2, padding="same")(X_4_0)
    X_3_1 = conv_block(layers.concatenate([X_3_0, u4_0]), 256)
    
    u3_1 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(X_3_1)
    X_2_2 = conv_block(layers.concatenate([X_2_0, X_2_1, u3_1]), 128)
    
    u2_2 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(X_2_2)
    X_1_3 = conv_block(layers.concatenate([X_1_0, X_1_1, X_1_2, u2_2]), 64)
    
    u1_3 = layers.Conv2DTranspose(32, 2, strides=2, padding="same")(X_1_3)
    X_0_4 = conv_block(layers.concatenate([X_0_0, X_0_1, X_0_2, X_0_3, u1_3]), 32)

    # --- Saídas ---
    output1 = layers.Conv2D(1, 1, activation="sigmoid", name="output_1")(X_0_1)
    output2 = layers.Conv2D(1, 1, activation="sigmoid", name="output_2")(X_0_2)
    output3 = layers.Conv2D(1, 1, activation="sigmoid", name="output_3")(X_0_3)
    output4 = layers.Conv2D(1, 1, activation="sigmoid", name="output_4")(X_0_4)

    if deep_supervision:
        return keras.Model(inputs, [output1, output2, output3, output4])
    else:
        return keras.Model(inputs, output4)

# Definir parâmetros e construir o modelo
PATCH_SIZE = 128
CHANNELS = 3
INPUT_SHAPE = (PATCH_SIZE, PATCH_SIZE, CHANNELS)
DEEP_SUPERVISION = True # Ativar a supervisão profunda

model = build_unet_plus_plus(INPUT_SHAPE, deep_supervision=DEEP_SUPERVISION)
model.summary()



## 6. Treinamento do Modelo


### 6.1. Função de Perda e Métricas

Para segmentação, uma combinação da perda Binary Cross-Entropy (BCE) e Dice Loss é muito eficaz. A BCE trata a classificação de cada pixel de forma independente, enquanto a Dice Loss avalia a sobreposição entre a predição e a máscara real, lidando bem com o desbalanceamento de classes (muito mais pixels de fundo do que de vasos).

In [ ]:
# %% Data Augmentation (Aumento de Dados)


BATCH_SIZE = 16
BUFFER_SIZE = len(train_img_patches) # Buffer para embaralhamento

def augment_data(image, mask):
    """
    Aplica transformações de aumento de dados em um par imagem-máscara.
    """
    # Empilhar imagem e máscara para aplicar a mesma transformação geométrica
    mask_float = tf.cast(mask, tf.float32)
    combined = tf.concat([image, mask_float], axis=-1)

    # Transformações Geométricas
    combined = tf.image.random_flip_left_right(combined)
    combined = tf.image.random_flip_up_down(combined)

    # Desempilhar a imagem e a máscara
    image = combined[:, :, :3]
    mask = combined[:, :, 3:]
    
    # Garantir que a máscara volte a ser binária
    mask = tf.cast(mask > 0.5, tf.float32)

    # Transformações de Cor (apenas na imagem)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    
    # Garantir que a imagem permaneça no intervalo [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, mask

def format_targets_for_deep_supervision(image, mask):
    """
    Formata a máscara para o dicionário esperado pelo modelo.
    """
    targets = {
        "output_1": mask,
        "output_2": mask,
        "output_3": mask,
        "output_4": mask,
    }
    return image, targets

# Criar o pipeline de dados de TREINAMENTO com aumento de dados
train_dataset = tf.data.Dataset.from_tensor_slices((train_img_patches, train_mask_patches))
train_dataset = (
    train_dataset.cache()
    .shuffle(BUFFER_SIZE)
    .map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Criar o pipeline de dados de VALIDAÇÃO (sem aumento de dados)
val_dataset = tf.data.Dataset.from_tensor_slices((val_img_patches, val_mask_patches))
val_dataset = (
    val_dataset
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Formatar os targets para supervisão profunda, se necessário
if DEEP_SUPERVISION:
    train_dataset = train_dataset.map(format_targets_for_deep_supervision)
    val_dataset = val_dataset.map(format_targets_for_deep_supervision)

print("Pipelines tf.data com aumento de dados foram criados com sucesso!")

In [ ]:
# Métrica de Dice Coefficient
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Função de perda (Dice Loss)
def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Perda combinada
def bce_dice_loss(y_true, y_pred):
    return keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

### 6.2. Compilação e Treinamento
Vamos compilar o modelo e iniciar o treinamento. Com a supervisão profunda, precisamos fornecer a máscara de treinamento para cada uma das quatro saídas do modelo.

In [ ]:
# Configurar o otimizador e a compilação
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

if DEEP_SUPERVISION:
    # A perda é aplicada a cada saída e somada
    losses = {
        "output_1": bce_dice_loss,
        "output_2": bce_dice_loss,
        "output_3": bce_dice_loss,
        "output_4": bce_dice_loss,
    }
    # Ponderar as perdas (a saída final é mais importante)
    loss_weights = {"output_1": 0.25, "output_2": 0.25,
                    "output_3": 0.25, "output_4": 1.0}

    # As métricas serão monitoradas apenas na saída final
    model.compile(optimizer=optimizer,
                  loss=losses,
                  loss_weights=loss_weights,
                  metrics={"output_4": [dice_coefficient, "accuracy"]})

else:
    model.compile(optimizer=optimizer,
                  loss=bce_dice_loss,
                  metrics=[dice_coefficient, "accuracy"])

# Callbacks para um treinamento mais robusto
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "unetpp_drive.keras", save_best_only=True, monitor="val_output_4_dice_coefficient", mode='max'),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_output_4_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    keras.callbacks.EarlyStopping(
        monitor="val_output_4_loss", patience=10, restore_best_weights=True, mode='min')
]

# Treinamento (NOVA CHAMADA model.fit)
EPOCHS = 60

history = model.fit(
    train_dataset,  # USA O NOVO PIPELINE DE TREINO
    epochs=EPOCHS,
    validation_data=val_dataset, # USA O NOVO PIPELINE DE VALIDAÇÃO
    callbacks=callbacks
)

## 7. Avaliação do Modelo
Após o treinamento, vamos avaliar o desempenho do modelo no conjunto de teste. A avaliação será feita nas imagens completas, o que requer a reconstrução das predições a partir dos patches.

### 7.1. Reconstrução da Imagem e Métricas

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score

def reconstruct_from_patches(patches, original_img_shape, stride=64):
    img_h, img_w = original_img_shape[0], original_img_shape[1]
    patch_size = patches.shape[1]
    
    reconstructed_img = np.zeros((img_h, img_w, 1))
    count_matrix = np.zeros((img_h, img_w, 1))
    
    patch_idx = 0
    for y in range(0, img_h - patch_size + 1, stride):
        for x in range(0, img_w - patch_size + 1, stride):
            if patch_idx < len(patches):
                reconstructed_img[y:y+patch_size, x:x+patch_size] += patches[patch_idx]
                count_matrix[y:y+patch_size, x:x+patch_size] += 1
                patch_idx += 1
            
    count_matrix[count_matrix == 0] = 1
    final_image = reconstructed_img / count_matrix
    return final_image

def evaluate_model(model, test_images, test_masks, test_fov_masks, patch_size=128, stride=64):
    all_metrics = {"dice": [], "accuracy": [], "sensitivity": [], "specificity": [], "auc": []}
    
    for i in range(len(test_images)):
        img = test_images[i]
        true_mask = test_masks[i]
        fov_mask = test_fov_masks[i].astype(bool).flatten()

        # Gerando todos os patches da imagem de teste, sem pular nenhum.
        img_h, img_w = img.shape[0], img.shape[1]
        test_patches_list = []
        for y in range(0, img_h - patch_size + 1, stride):
            for x in range(0, img_w - patch_size + 1, stride):
                patch = img[y:y+patch_size, x:x+patch_size]
                test_patches_list.append(patch)
        
        test_patches = np.array(test_patches_list)

        if len(test_patches) == 0:
            print(f"Aviso: Nenhum patch gerado para a imagem {i}, pulando.")
            continue

        if DEEP_SUPERVISION:
            preds = model.predict(test_patches, verbose=0)[-1]
        else:
            preds = model.predict(test_patches, verbose=0)

        pred_mask = reconstruct_from_patches(preds, img.shape, stride=stride)
        pred_mask_flat = pred_mask.flatten()
        pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)
        pred_mask_bin_flat = pred_mask_bin.flatten()
        true_mask_flat = true_mask.flatten()
        true_mask_fov = true_mask_flat[fov_mask]
        pred_mask_bin_fov = pred_mask_bin_flat[fov_mask]
        pred_mask_fov = pred_mask_flat[fov_mask]

        if len(np.unique(true_mask_fov)) < 2:
            print(f"Aviso: Máscara da imagem {i} não tem ambas as classes após filtro FOV. Pulando.")
            continue
        
        tn, fp, fn, tp = confusion_matrix(true_mask_fov, pred_mask_bin_fov, labels=[0, 1]).ravel()
        
        dice = (2. * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
        sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        acc = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

        all_metrics["dice"].append(dice)
        all_metrics["accuracy"].append(acc)
        all_metrics["sensitivity"].append(sens)
        all_metrics["specificity"].append(spec)
        all_metrics["auc"].append(roc_auc_score(true_mask_fov, pred_mask_fov))

    avg_metrics = {key: np.mean(val) for key, val in all_metrics.items()}
    return avg_metrics

In [ ]:
# Recarregando os dados de treino completos para garantir consistência
train_images, train_masks, train_fov_masks = load_data(TRAIN_IMG_DIR, TRAIN_MASK_DIR, TRAIN_FOV_DIR)

# Separar treino e validação (incluindo o fov_masks agora)
X_train, X_val, y_train, y_val, fov_train, fov_val = train_test_split(
    train_images, train_masks, train_fov_masks, test_size=0.2, random_state=42 # Aumentei para 20% para uma validação mais robusta
)

# Criar patches para treinamento
train_img_patches, train_mask_patches = create_patches(X_train, y_train)


In [ ]:
# Carregar o melhor modelo salvo
model = keras.models.load_model("unetpp_drive.keras", custom_objects={
    'bce_dice_loss': bce_dice_loss, 
    'dice_coefficient': dice_coefficient
})

print("--- Iniciando Avaliação Quantitativa no Conjunto de VALIDAÇÃO ---")

# Esta linha CHAMA a função que definimos antes e calcula as métricas
validation_metrics = evaluate_model(model, X_val, y_val, fov_val)

print("\n--- Métricas Finais no Conjunto de Validação (média) ---")
for metric, value in validation_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}")

### 7.2. Visualização dos Resultados
Visualizar as predições é a melhor forma de ter uma avaliação qualitativa.

In [ ]:
# O objetivo é comparar a imagem, a máscara real e a predição.
# Faremos isso no CONJUNTO DE VALIDAÇÃO, pois para ele temos as máscaras de gabarito.

print("\n--- Iniciando Predição e Visualização Comparativa no Conjunto de VALIDAÇÃO ---")

# A função de plotagem permanece a mesma, ela já está correta.
# Apenas vamos renomeá-la para maior clareza.
def plot_validation_predictions(model, images_to_plot, true_masks, num_samples=3):
    """
    Plota a imagem original, a máscara real (gabarito) e a máscara predita.
    """
    # Garante que não tentaremos plotar mais amostras do que as disponíveis
    num_samples = min(num_samples, len(images_to_plot))

    for i in range(num_samples):
        img = images_to_plot[i]
        true_mask = true_masks[i]

        # Lógica para criar patches e prever
        img_h, img_w = img.shape[0], img.shape[1]
        patch_size = 128
        stride = 64

        test_patches_list = []
        for y in range(0, img_h - patch_size + 1, stride):
            for x in range(0, img_w - patch_size + 1, stride):
                test_patches_list.append(img[y:y+patch_size, x:x+patch_size])

        test_patches_np = np.array(test_patches_list)

        if test_patches_np.shape[0] == 0:
            print(f"Nenhum patch gerado para a imagem de validação {i}. Pulando.")
            continue

        # Fazer predições nos patches
        if DEEP_SUPERVISION:
            preds = model.predict(test_patches_np, verbose=0)[-1]
        else:
            preds = model.predict(test_patches_np, verbose=0)

        # Reconstruir a máscara completa
        pred_mask = reconstruct_from_patches(preds, img.shape, stride=stride)
        pred_mask_bin = (pred_mask > 0.5).astype(np.uint8)

        # Plotagem comparativa
        plt.figure(figsize=(18, 6))

        # Plot 1: Imagem de Validação Original
        plt.subplot(1, 3, 1)
        plt.title(f"Imagem de Validação #{i+1}")
        plt.imshow(img)
        plt.axis('off')

        # Plot 2: Máscara Real (Gabarito)
        plt.subplot(1, 3, 2)
        plt.title("Máscara Real (Gabarito)")
        plt.imshow(true_mask.squeeze(), cmap='gray') # .squeeze() remove dimensões extras
        plt.axis('off')

        # Plot 3: Máscara Predita
        plt.subplot(1, 3, 3)
        plt.title("Máscara Predita pelo Modelo")
        plt.imshow(pred_mask_bin.squeeze(), cmap='gray')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Chamamos a função passando os dados de VALIDAÇÃO (X_val, y_val)
plot_validation_predictions(model, X_val, y_val, num_samples=6)

In [ ]:
import pandas as pd

# 'validation_metrics' é o dicionário que obtivemos na célula de avaliação
# Se não o tiver mais, rode a avaliação novamente.

# Criar um DataFrame do pandas a partir do dicionário
df_metrics = pd.DataFrame([validation_metrics])

# Transpor a tabela para que as métricas fiquem nas linhas
df_metrics = df_metrics.T.reset_index()

# Renomear as colunas para maior clareza
df_metrics.columns = ['Métrica', 'Valor']

# Arredondar os valores para 4 casas decimais
df_metrics['Valor'] = df_metrics['Valor'].round(4)

print("Tabela de Desempenho do Modelo U-Net++ (no conjunto de validação)")
display(df_metrics)

In [ ]:
import matplotlib.pyplot as plt

# Acessar os dados do histórico de treinamento
history_dict = history.history

# Perda (Loss)
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# Dice Coefficient (lembre-se que o nome da métrica é 'output_4_dice_coefficient' por causa da supervisão profunda)
dice = history_dict['output_4_dice_coefficient']
val_dice = history_dict['val_output_4_dice_coefficient']

epochs = range(1, len(loss) + 1)

# Criar a figura com dois subplots
plt.figure(figsize=(14, 5))

# Subplot 1: Gráfico de Perda
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo-', label='Perda de Treino')
plt.plot(epochs, val_loss, 'ro-', label='Perda de Validação')
plt.title('Perda de Treino vs. Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda (Loss)')
plt.grid(True)
plt.legend()

# Subplot 2: Gráfico de Dice Coefficient
plt.subplot(1, 2, 2)
plt.plot(epochs, dice, 'bo-', label='Dice de Treino')
plt.plot(epochs, val_dice, 'ro-', label='Dice de Validação')
plt.title('Dice Coefficient de Treino vs. Validação')
plt.xlabel('Épocas')
plt.ylabel('Dice Coefficient')
plt.grid(True)
plt.legend()

plt.suptitle('Evolução do Treinamento do Modelo', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Vamos pegar uma imagem do conjunto de validação para gerar a curva
img_exemplo = X_val[0]
mask_exemplo = y_val[0]
fov_exemplo = fov_val[0]

# Gerar patches da imagem de exemplo
img_h, img_w = img_exemplo.shape[0], img_exemplo.shape[1]
patch_size = 128
stride = 64
patches_exemplo_list = []
for y in range(0, img_h - patch_size + 1, stride):
    for x in range(0, img_w - patch_size + 1, stride):
        patches_exemplo_list.append(img_exemplo[y:y+patch_size, x:x+patch_size])
patches_exemplo_np = np.array(patches_exemplo_list)

# Fazer a predição
if DEEP_SUPERVISION:
    preds_exemplo = model.predict(patches_exemplo_np, verbose=0)[-1]
else:
    preds_exemplo = model.predict(patches_exemplo_np, verbose=0)

# Reconstruir a predição
pred_mask_exemplo = reconstruct_from_patches(preds_exemplo, img_exemplo.shape, stride=stride)

# Aplicar a máscara de FOV (campo de visão)
true_labels = mask_exemplo.flatten()[fov_exemplo.flatten().astype(bool)]
pred_scores = pred_mask_exemplo.flatten()[fov_exemplo.flatten().astype(bool)]

# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(true_labels, pred_scores)
roc_auc = auc(fpr, tpr)

# Plotar o gráfico
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (área = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classificador Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)')
plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)')
plt.title('Curva ROC para uma Imagem de Exemplo')
plt.grid(True)
plt.legend(loc="lower right")
plt.show()